In [53]:
# Dependencies

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup as bs
from selenium.webdriver.support.ui import WebDriverWait
import re
import pandas as pd
import os
import time
from splinter import Browser
from urllib.request import urlopen
import requests

In [54]:
# Website URL

url = "https://www.otcmarkets.com/market-activity/current-market/ALL/active/dollarVolume"

In [55]:
# Website URL
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
html = "https://www.otcmarkets.com/market-activity/current-market/ALL/active/dollarVolume"
browser.visit(html)
html = browser.html
soup = BeautifulSoup(html,'html.parser')

In [56]:
# Website hmtl page title (Wait for 6 Secs for the browser to open)

title = soup.title.text
print(title)

OTC Markets | Market Activity - Current Market


In [57]:
# Print formatted version of the soup

print(soup.prettify())

<html class="wf-nunitosans-n7-active wf-nunitosans-n4-active wf-sourceserifpro-n7-active wf-nunitosans-n2-active wf-nunitosans-i7-active wf-sourceserifpro-n4-active wf-nunitosans-i4-active wf-nunitosans-i2-active wf-active" lang="en">
 <head>
  <script src="https://cdn.krxd.net/userdata/get?pub=9427dd0d-835c-471c-a5db-ab01ae8a681c&amp;kxfp=d15d595b90cfb2fb169104df84ac0ce94180d6fb&amp;technographics=1&amp;callback=Krux.ns.investingchannelinc.kxjsonp_userdata" type="text/javascript">
  </script>
  <script src="https://beacon.krxd.net/optout_check?callback=Krux.ns.investingchannelinc.kxjsonp_optOutCheck" type="text/javascript">
  </script>
  <script src="https://js-agent.newrelic.com/nr-spa-1071.min.js">
  </script>
  <script src="https://securepubads.g.doubleclick.net/gpt/pubads_impl_rendering_2019041801.js?21063675">
  </script>
  <script async="" src="https://gads.pubmatic.com/AdServer/AdCallAggregator?SAVersion=1000&amp;js=1&amp;sec=1&amp;kltstamp=2019-5-2%2018%3A49%3A25&amp;timezone=

In [58]:
# Extract Most Active Stocks html class

most_active_stocks = soup.findAll("div", {"class": "_12doApv4EC"})
print(most_active_stocks)

[<div class="_12doApv4EC">Displaying 25 of 18513 Most Active</div>]


In [59]:
# Extract Most Active Stocks Number List

most_active_stocks_List = []

for x in most_active_stocks:
       most_active_stocks_List.append(x.text[17:22])
print(most_active_stocks_List)  

['18513']


In [60]:
# Extract Most Active Stocks Number Only

most_active_stocks_no = int(most_active_stocks_List[0])
print (most_active_stocks_no)

18513


In [61]:
# Request to open the browser and wait for 6 secs

driver = webdriver.Chrome('/usr/local/bin/chromedriver')
driver.get(url)
#time.sleep(6)

In [ ]:
# Find all div class "_2_IqcDGekT" to look for the TIER (Logo Column)
## Loop through the Table and append to the OTC
# Click on the More button to open all the stock pages (More Button 'Class="_12doApv4EC"')  
### Add "most_active_stocks_no" instead of "5" (Each page has 25 Records, Range 5 stops at 150 records) 

OTC_list = []
TIER_list = []


tier_column = soup.findAll("div", {"class": "_2_IqcDGekT"})

table_rows = soup.find_all('tr')


for t in tier_column:
        TIER_list.append(t.img["src"][12:14])

for tr in table_rows:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            OTC_list.append(row)

    
for  x  in range(0,most_active_stocks_no):

        
        for tr in table_rows:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            OTC_list.append(row)
            
        for t in tier_column:
            TIER_list.append(t.img["src"][12:14])

        python_button = driver.find_element_by_class_name('_2sFaw3zGf1')
        python_button.click()
        
        time.sleep(5)
        
        
        if x == most_active_stocks_no:
            break
    

In [ ]:
# TIER DataFrame
TIER = pd.DataFrame(TIER_list, columns=["TIER"])
TIER.head()

In [ ]:
# Start TIER index at 1 instead of Zero 
TIER.index += 1
TIER.head()

In [ ]:
# Convert the OTC List into Dataframe
OTC = pd.DataFrame(OTC_list, columns=["SYMBOL", "PRICE", "% CHANGE", "$ VOL", "SHARE_VOL", "TRADES"])
OTC.head()

In [ ]:
# Dropping the first OTC df row
OTC = OTC.iloc[1:]
OTC.head()

In [ ]:
TIER_COUNT = TIER.count()
TIER_COUNT

In [ ]:
OTC_COUNT = OTC.count()
OTC_COUNT

In [ ]:
OTC_Markets = pd.merge(TIER, OTC, left_index=True, right_index=True)
OTC_Markets.head()

In [ ]:
# Remove OTC Pink (PS), which is the lowest and most speculative tier 
OTC_Markets = OTC_Markets[OTC_Markets.TIER != 'ps']
OTC_Markets.head()

In [ ]:
# Reset Index
OTC_Markets = OTC_Markets.reset_index(drop=True)
OTC_Markets.index += 1
OTC_Markets.head()

In [ ]:
import pandas as pd
from pymongo import MongoClient
import json
import os
import datetime

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.otc_markets
collection = db.most_active

In [ ]:
records = json.loads(OTC_Markets.T.to_json()).values()
db.most_active.insert_many(OTC_Markets.to_dict('records'))